# Correr Experimentos

## Includes

In [1]:
%load_ext autoreload
%autoreload 2

import subprocess
import numpy as np
import pandas as pd
from IPython.display import display, clear_output

In [2]:
!cd .. && make

g++ -c -o ../obj/main.o main.cpp -Wall -Wno-unused-parameter -Wextra -O2 -pedantic -std=c++14 -I../include
g++ -c -o ../obj/solver.o solver.cpp -Wall -Wno-unused-parameter -Wextra -O2 -pedantic -std=c++14 -I../include
g++ -o jt ../obj/main.o ../obj/solver.o -Wall -Wno-unused-parameter -Wextra -O2 -pedantic -std=c++14 -I../include


In [3]:
BIN = "../jt"
def run(metodo: str, archivo: str) -> float:
    completed_process = subprocess.run([BIN, metodo, archivo], capture_output=True)
    
    completed_process.check_returncode()
    return float(completed_process.stderr)

In [5]:
df_fuerza_bruta = pd.read_csv("instancias/fuerza-bruta/indice.csv")
df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-bt/indice.csv")
df_peor_caso_bt = pd.read_csv("instancias/peor-caso-bt/indice.csv")
df_sin_superposicion = pd.read_csv("instancias/sin-superposicion/indice.csv")
df_alta_superposicion = pd.read_csv("instancias/alta-superposicion/indice.csv")
df_muchos_productos = pd.read_csv("instancias/muchos-productos/indice.csv")

experimentos = []

##  Experimento 1: Complejidad de Fuerza bruta

In [6]:
METODO = "BF"
for _, row in df_fuerza_bruta.iterrows():
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : METODO,
        "n"       : row["n"],
        "R"       : row["R"],
    })

## Experimento 2: Complejidad backtracking

Mejor y peor caso con ambas podas:

In [7]:
METODO = "BT"
for _, row in df_mejor_caso_bt.iterrows():
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : METODO,
        "n"       : row["n"],
        "R"       : row["R"],
    })

for _, row in df_peor_caso_bt.iterrows():
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : METODO,
        "n"       : row["n"],
        "R"       : row["R"],
    })

## Experimento 4: Comparación backtracking--programción dinámica

In [10]:
for _, row in df_sin_superposicion.iterrows():
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : "DP",
        "n"       : row["n"],
        "R"       : row["R"],
    })
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : "BT",
        "n"       : row["n"],
        "R"       : row["R"],
    })
for _, row in df_alta_superposicion.iterrows():
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : "DP",
        "n"       : row["n"],
        "R"       : row["R"],
    })
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : "BT",
        "n"       : row["n"],
        "R"       : row["R"],
    })

## Experimento 5: Programción dinámica con muchos productos

In [11]:
METODO = "DP"
for _, row in df_muchos_productos.iterrows():
    experimentos.append({
        "dataset" : row["dataset"],
        "archivo" : row["filename"],
        "metodo"  : METODO,
        "n"       : row["n"],
        "R"       : row["R"],
    })

## Corremos los experimentos

In [13]:
cols = ["dataset", "n", "R", "metodo", "tiempo"];
res = []
numero = 1
T = 5

for exp in experimentos:
    
    clear_output(wait=True)
    display(f'Experimento: {numero}/{len(experimentos)}')
    numero += 1
    
    tiempos = []
    for _ in range(0, T):
        tiempos.append(run(exp["metodo"], exp["archivo"]))
    tiempo_avg = np.median(tiempos)
    
    res.append([exp["dataset"], exp["n"], exp["R"], exp["metodo"], tiempo_avg])
        
df_res = pd.DataFrame(res, columns=cols)
df_res.to_csv("resultados.csv", index=False, header=True)

'Experimento: 411/411'